<a href="https://colab.research.google.com/github/abdul-criana/Work-like-a-hell/blob/main/Wine_and_Food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
import pandas as pd

# Import Dataset

In [4]:
uploaded = files.upload()
uploaded = files.upload()

Saving Food_Recipe.csv to Food_Recipe.csv


Saving WineDataset.csv to WineDataset (1).csv


In [5]:
wine_df = pd.read_csv("WineDataset.csv")
food_df = pd.read_csv("Food_Recipe.csv")

In [6]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

# Wine Dataset: Cleaning & Normalizing

In [7]:
# Drop rows with missing descriptions or grape (key for flavor)
wine_df = wine_df.dropna(subset=["Description", "Grape"])

In [8]:
# Fill missing optional fields with empty string
for col in ["Secondary Grape Varieties", "Characteristics", "Style"]:
    wine_df[col] = wine_df[col].fillna("")

# Combine important fields into a single 'wine_text' column
wine_df["wine_text"] = (
    wine_df["Grape"] + " " +
    wine_df["Secondary Grape Varieties"] + " " +
    wine_df["Characteristics"] + " " +
    wine_df["Style"] + " " +
    wine_df["Description"]
)

In [9]:
# Keep only needed columns
wine_df_clean = wine_df[["Title", "wine_text"]].drop_duplicates().reset_index(drop=True)

print("Wine dataset ready:", wine_df_clean.shape)
wine_df_clean.head(3)

Wine dataset ready: (1272, 2)


,Title,wine_text
0,"The Guv'nor, Spain","Tempranillo Vanilla, Blackberry, Blackcurrant..."
1,Bread & Butter 'Winemaker's Selection' Chardon...,"Chardonnay Vanilla, Almond, Coconut, Green Ap..."
2,"Oyster Bay Sauvignon Blanc 2022, Marlborough","Sauvignon Blanc Tropical Fruit, Gooseberry, G..."


#  Food Dataset: Cleaning & Normalizing

In [10]:
# Drop rows with missing ingredients or description
food_df = food_df.dropna(subset=["ingredients_name", "description"])

In [11]:
# Fill optional columns with empty string
for col in ["cuisine", "course", "diet"]:
    food_df[col] = food_df[col].fillna("")

# Combine important fields into one 'food_text' column
food_df["food_text"] = (
    food_df["ingredients_name"] + " " +
    food_df["cuisine"] + " " +
    food_df["course"] + " " +
    food_df["diet"] + " " +
    food_df["description"]
)

In [12]:
# Keep only needed columns
food_df_clean = food_df[["name", "food_text"]].drop_duplicates().reset_index(drop=True)

print("Food dataset ready:", food_df_clean.shape)
food_df_clean.head(3)

Food dataset ready: (3995, 2)


,name,food_text
0,Mulakootal Recipe (Vegetables In Coconut Gravy),"Yellow Moong Dal (Split), Mixed vegetables, Tu..."
1,Parshe Macher Jhal Recipe - Mullet In Mustard ...,"Parshe, Kasundi mustard sauce, Tomatoes, Green..."
2,Chocolate Chip & Raisin Rolls Recipe,"Whole Wheat Flour, All Purpose Flour (Maida), ..."


# Embedding the Data

In [13]:
from sentence_transformers import SentenceTransformer

In [14]:
# Load pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode wine descriptions
wine_embeddings = model.encode(
    wine_df_clean["wine_text"].tolist(),
    show_progress_bar=True,
    convert_to_tensor=True
)

# Encode food descriptions
food_embeddings = model.encode(
    food_df_clean["food_text"].tolist(),
    show_progress_bar=True,
    convert_to_tensor=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

Batches:   0%|          | 0/125 [00:00<?, ?it/s]

In [15]:
import torch

# Save the embeddings and original data (optional)
torch.save(wine_embeddings, "wine_embeddings.pt")
torch.save(food_embeddings, "food_embeddings.pt")

wine_df_clean.to_csv("wine_clean.csv", index=False)
food_df_clean.to_csv("food_clean.csv", index=False)

# Matching Wine with Food

In [16]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [17]:
def recommend_wine_for_food(food_index, top_k=5):
    # Get the embedding for the selected food
    food_vec = food_embeddings[food_index].unsqueeze(0)

    # Compute similarity with all wine embeddings
    similarities = cosine_similarity(food_vec, wine_embeddings)[0]

    # Get top K wine indices
    top_indices = similarities.argsort()[-top_k:][::-1]

    print(f"\n🥘 Dish: {food_df_clean.loc[food_index, 'name']}")
    print(f"🔍 Ingredients: {food_df_clean.loc[food_index, 'food_text'][:100]}...")

    print("\n🍷 Top Wine Matches:")
    for idx in top_indices:
        print(f"✅ {wine_df_clean.loc[idx, 'Title']} — score: {similarities[idx]:.4f}")


In [18]:
def recommend_food_for_wine(wine_index, top_k=5):
    # Get the embedding for the selected wine
    wine_vec = wine_embeddings[wine_index].unsqueeze(0)

    # Compute similarity with all food embeddings
    similarities = cosine_similarity(wine_vec, food_embeddings)[0]

    # Get top K food indices
    top_indices = similarities.argsort()[-top_k:][::-1]

    print(f"\n🍷 Wine: {wine_df_clean.loc[wine_index, 'Title']}")
    print(f"🔍 Profile: {wine_df_clean.loc[wine_index, 'wine_text'][:100]}...")

    print("\n🥘 Top Food Matches:")
    for idx in top_indices:
        print(f"✅ {food_df_clean.loc[idx, 'name']} — score: {similarities[idx]:.4f}")


In [19]:
# recommend_food_for_wine(wine_index=17)  # pick any wine index you want

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def recommend_wine_for_food(food_index, top_k=5):
    # Get the embedding for the selected food
    food_vec = food_embeddings[food_index].unsqueeze(0).cpu() # Move to CPU

    # Compute similarity with all wine embeddings
    similarities = cosine_similarity(food_vec.numpy(), wine_embeddings.cpu().numpy())[0] # Move to CPU and convert to NumPy

    # Get top K wine indices
    top_indices = similarities.argsort()[-top_k:][::-1]

    print(f"\n🥘 Dish: {food_df_clean.loc[food_index, 'name']}")
    print(f"🔍 Ingredients: {food_df_clean.loc[food_index, 'food_text'][:100]}...")

    print("\n🍷 Top Wine Matches:")
    for idx in top_indices:
        print(f"✅ {wine_df_clean.loc[idx, 'Title']} — score: {similarities[idx]:.4f}")

In [21]:
recommend_wine_for_food(food_index=42)


🥘 Dish: Kala Chana Pilaf Recipe
🔍 Ingredients: Basmati Rice, Kala Chana (Brown Chickpeas), Onion, Tomato, Green Chillies, Ginger Garlic Paste, Bay ...

🍷 Top Wine Matches:
✅ Plum & Pepper Barossa Shiraz 2017 — score: 0.4454
✅ Yellow Tail Chardonnay 2021/22 — score: 0.4366
✅ Wynns Coonawarra Estate Shiraz 2020/21, Coonawarra — score: 0.4111
✅ Making Tracks Shiraz 2019/20, South Australia — score: 0.4107
✅ Yellow Tail Shiraz 2020/21, Australia — score: 0.4040


In [24]:
def get_wine_recommendations(dish_name, top_k=5):
    try:
        # Find the food index based on dish name
        food_index = food_df_clean[food_df_clean['name'] == dish_name].index[0]
        recommend_wine_for_food(food_index, top_k)
    except IndexError:
        print(f"Dish '{dish_name}' not found in the database.")

# Example usage
dish = input("Enter a dish: ")
get_wine_recommendations(dish)

Enter a dish: Curried Poor Curry

🥘 Dish: Curried Poor Curry
🔍 Ingredients: Potato (Aloo), Homemade tomato puree, Onion, Ginger, Garlic, Turmeric powder (Haldi), Oil, Salt, Cor...

🍷 Top Wine Matches:
✅ Torre Mora ‘Cauru’ Etna Rosso DOC 2020/21 — score: 0.4652
✅ Toast & Honey Chardonnay 2021, California — score: 0.4571
✅ Cantine Maschio Extra Dry Prosecco DOC — score: 0.4526
✅ Domini Veneti Soave Classico DOC 2021/22 — score: 0.4443
✅ Pasqua 'PassioneSentimento' Passimento Rosso 2020/21, Veneto — score: 0.4380
